In [1]:
from __future__ import annotations

In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    Transmon,
)

SAMPLING_PERIOD = 0.5
qp.set_sampling_period(SAMPLING_PERIOD)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# units: ns, GHz
transmon_dimension = 3
control_frequency = 7.456
target_frequency = 8.654
anharmonicity = -0.333
relaxation_rate = 5e-5
dephasing_rate = 5e-5
coupling_strength = 0.01

qubits = [
    Transmon(
        label="Q0",
        dimension=transmon_dimension,
        frequency=control_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
    Transmon(
        label="Q1",
        dimension=transmon_dimension,
        frequency=target_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
]

couplings = [
    Coupling(
        pair=("Q0", "Q1"),
        strength=coupling_strength,
    ),
]

system = QuantumSystem(
    objects=qubits,
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [5]:
control_qubit = qubits[0]
target_qubit = qubits[1]

drag_duration = 16  # ns

In [6]:
def drag_pi_pulse(
    duration: float,
    beta: float,
) -> qp.Pulse:
    pulse = qp.Drag(
        duration=duration,
        amplitude=1,
        beta=beta,
    )
    norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
    pulse = pulse.scaled(norm_factor)
    return pulse

In [7]:
alpha = 2 * np.pi * control_qubit.anharmonicity
beta = -0.5 / alpha
pi_pulse = drag_pi_pulse(
    duration=drag_duration,
    beta=beta,
)
pi_pulse.plot(divide_by_two_pi=True)

drag_amplitude = np.max(pi_pulse.real)
drag_amplitude

0.30841775708529257

In [8]:
result = simulator.mesolve(
    controls=[
        Control(
            target=control_qubit.label,
            frequency=control_qubit.frequency,
            waveform=pi_pulse,
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere(control_qubit.label)
result.show_last_population(control_qubit.label)

<IPython.core.display.Javascript object>

|0⟩: 0.000626
|1⟩: 0.999359
|2⟩: 0.000015


In [9]:
results = []
beta_range = np.linspace(-1, 1, 101)
for beta in beta_range:
    alpha = 2 * np.pi * anharmonicity
    pulse = qp.Drag(
        duration=drag_duration,
        amplitude=1,
        beta=beta,
    )
    norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
    pulse = pulse.scaled(norm_factor)
    result = simulator.mesolve(
        controls=[
            Control(
                target=control_qubit.label,
                frequency=control_qubit.frequency,
                waveform=qp.PulseSequence([
                    pulse,
                    pulse.scaled(-1),
                ])
            )
        ],
        initial_state=simulator.system.ground_state,
    )
    results.append(result)

In [10]:
results[50].display_bloch_sphere(control_qubit.label)
results[60].display_bloch_sphere(control_qubit.label)
results[70].display_bloch_sphere(control_qubit.label)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
from qubex.analysis import plot_xy

e_x = np.array(
    [result.get_bloch_vectors(control_qubit.label)[-1][0] for result in results]
)

plot_xy(beta_range, e_x)

In [12]:
from scipy.optimize import root_scalar

e_x_fine = lambda x: np.interp(x, beta_range, e_x)

result = root_scalar(e_x_fine, bracket=[beta_range[0], beta_range[-1]])
drag_beta = result.root
drag_beta

0.22798109202529168

In [13]:
def drag_objective_func(x):
    amplitude, beta = x
    result = simulator.mesolve(
        controls=[
            Control(
                target=control_qubit.label,
                frequency=control_qubit.frequency,
                waveform=qp.Drag(
                    duration=drag_duration,
                    amplitude=amplitude,
                    beta=beta,
                )
            )
        ],
        initial_state=simulator.system.ground_state,
    )
    state = result.get_bloch_vectors(control_qubit.label)[-1]
    target = np.array([0, 0, -1])
    return np.linalg.norm(state - target)

In [14]:
drag_objective_func([drag_amplitude, drag_beta])

0.02304666083941402

In [15]:
import cma

initial_guess = [
    drag_amplitude,
    drag_beta,
]

es = cma.CMAEvolutionStrategy(
    initial_guess,
    0.01,
    {
        "seed": 42,
        "ftarget": 1e-6,
    },
)

es.optimize(drag_objective_func)

es.result.xbest

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=42, Mon Nov 25 17:18:53 2024)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 2.705345061821724e-02 1.0e+00 8.64e-03  8e-03  8e-03 0:00.1
    2     12 7.954579650098379e-03 1.1e+00 1.01e-02  7e-03  1e-02 0:00.7
    3     18 2.796342820385240e-02 1.8e+00 1.02e-02  6e-03  1e-02 0:01.0
   24    144 1.085935924826973e-03 1.3e+01 1.05e-03  4e-05  6e-04 0:04.0
   49    294 1.071408860922842e-03 2.9e+01 9.24e-05  4e-07  1e-05 0:08.1
   81    486 1.071321468412150e-03 2.4e+00 3.27e-06  8e-09  1e-08 0:13.1
  100    600 1.071319662308501e-03 3.5e+01 1.64e-06  6e-09  1e-08 0:15.9
  149    894 1.071316958876544e-03 7.2e+02 1.32e-06  1e-09  2e-09 0:22.8


array([0.3107031 , 0.22805689])

In [16]:
drag_objective_func(es.result.xbest)

0.0010713169586918393

In [17]:
pi_pulse = qp.Drag(
    duration=drag_duration,
    amplitude=es.result.xbest[0],
    beta=es.result.xbest[1],
)
pi_pulse.plot(divide_by_two_pi=True)

result = simulator.mesolve(
    controls=[
        Control(
            target=control_qubit.label,
            frequency=control_qubit.frequency,
            waveform=pi_pulse,
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere(control_qubit.label)
result.show_last_population(control_qubit.label)

<IPython.core.display.Javascript object>

|0⟩: 0.000527
|1⟩: 0.999456
|2⟩: 0.000017
